In [1]:
%%HTML
<link rel="stylesheet" href="https://doc.splicemachine.com/jupyter/css/custom.css">

In [ ]:
%%scala
%%spark <> --noUI
import java.net.InetAddress
val driver_host = InetAddress.getLocalHost.getHostAddress
SparkSession.builder()
	.appName("jt1test2")
	.master("k8s://https://kubernetes.default.svc.cluster.local:443")
	.config("spark.kubernetes.container.image", "splicemachine/sm_k8_spark:0.0.4")
	.config("spark.executor.instances", "2")
	.config("spark.submit.deployMode", "cluster")
	.config("spark.submit.deployMode", "cluster")
	.config("spark.driver.extraClassPath", "/opt/spark/conf:/opt/spark/jars/*")
	.config("spark.executor.extraClassPath", "./:/opt/hbase/conf:/opt/splicemachine/lib/*:/opt/spark/jars/*:/opt/hbase/lib/*")
	.config("splice.spark.executor.extraLibraryPath", "/opt/native")
	.config("spark.files", "/opt/spark/conf/hbase-site.xml,/opt/spark/conf/core-site.xml,/opt/spark/conf/hdfs-site.xml,/opt/spark/jars/hbase_sql-2.8.0.1926-cdh5.14.0.jar")
	.config("spark.kubernetes.authenticate.caCertFile", "/var/run/secrets/kubernetes.io/serviceaccount/ca.crt")
	.config("spark.kubernetes.authenticate.oauthTokenFile", "/var/run/secrets/kubernetes.io/serviceaccount/token")
	.config("spark.driver.host", driver_host)
	.config("spark.kubernetes.authenticate.driver.serviceAccountName", "spark")



# Splice Transactions with Spark and JDBC

Now we'll get a glimpse into the transactional nature of Splice Machine.  We'll switch to the `%spark` interpreter, which uses the Scala programming language; this allows us to easily demonstrate a clean way to programmatically interact with Splice Machine via JDBC. In this demonstration, we'll:

<ul class="italic">
    <li>Run setup code to create our variables and define a simple Spark procedure named <code>checkBalance</code>.</li>
    <li>Use JDBC calls to set and check values from the database.</li>
    <li>Show the use of <code>commit</code> and <code>rollback</code> to control transactional persistence of our updates.</li>
</ul>

## Run Setup Code and Define our Procedure

The first step is to run the next cell, which displays a form field. In that form, enter the JDBC URL, userId, and password you use to connect to Splice Machine, then click the *Go!* button.

In the next cell, insert your JDBC URL, along with your user ID and password, in the <code>defaultJDBCURL</code> field:

In [3]:
%%scala 
println("Please copy and paste your JDBC URL. You can find it at the bottom right of your cluster dashboard")
val form : EasyForm = new com.twosigma.beakerx.scala.easyform.EasyForm("JDBC Input")
form.addTextField("jdbc")
form.put("jdbc", "Enter your JDBC Url here")
form.addButton("Go!")
form


Please copy and paste your JDBC URL. You can find it at the bottom right of your cluster dashboard


Next we connect to the database and then define a simple procedure, `checkBalance`, which checks and *pretty-prints* the current balance for a supplier:

In [ ]:
%%scala 
import java.sql.{DriverManager,Connection}
var conn = DriverManager.getConnection(form.get("jdbc"))
conn.setAutoCommit(false)
var statement = conn.createStatement()

def checkBalance(suppKey: Integer) : Unit = {
    var resSet = statement.executeQuery("SELECT S_ACCTBAL FROM TPCH1.SUPPLIER WHERE S_SUPPKEY = " + suppKey.toString())
    while ( resSet.next() ) {
        var bl = resSet.getString("S_ACCTBAL")
        println("================ Current account balance: " + bl + " ====================");
    }
}


Now we can check the balance easily at any time with the `checkBalance` call, passing in the supply key as a parameter. For example:

In [ ]:
%%scala 
checkBalance(6517)

## Committing a Transaction

Our setup code created two useful variables we can now employ:

<table class="splicezepNoBorder">
   <tbody>
      <tr>
          <td><code>statement</code></td>
          <td>The standard JDBC Statement object, on which we run functions such as <code>executeUpdate</code>.</td>
      </tr>
      <tr>
          <td><code>conn</code></td>
          <td>A standard JDBC Connection object, on which we can run <code>commit</code> and <code>rollback</code>.</td>
      </tr>
    </tbody>
</table>

The first example shows an update statement which, upon committing, retains the value to which it is set:

In [ ]:
%%scala 
statement.executeUpdate("update TPCH1.SUPPLIER SET S_ACCTBAL = 10000 WHERE S_SUPPKEY = 6517");
checkBalance(6517);
conn.commit();
checkBalance(6517);

## Rolling Back a Transaction

If we choose to do a rollback, the value returns to the previous value before the transactional context, as shown in the next example:


In [ ]:
%%scala 
statement.executeUpdate("update TPCH1.SUPPLIER SET S_ACCTBAL = 0 WHERE S_SUPPKEY = 6517")
checkBalance(6517);
conn.rollback();
checkBalance(6517);


## Where to Go Next

The next notebook in this presentation walks you through <a href="./3.5%20Creating%20Applications.ipynb">creating applications with Splice Machine.</a>
